In [88]:
import cherrypy
import spacy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import wordnet

nlp = spacy.load('en')

In [90]:
syns = wordnet.synsets("program")
print(syns[0].name())

LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/Users/Markos/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [63]:
def return_lang_part(fext, nlp, part_of_the_language):
    text='I can '+fext
    text_array=np.array(text.split())
    doc = nlp(text)
    if part_of_the_language == 'VERB':
        result = np.array([str(token) for token in doc if token.pos_ == part_of_the_language and token.tag_ == 'VB'])
        if (len(result)>0):
            s=''
            s+=result[0]
            index = np.argwhere(text_array==result[0])
            numptext=np.array(text_array)
            try:
                if text_array[index+1]=='in':
                    s+=' in'
                if text_array[index+1]=='up':
                    s+=' up'  
            except:
                pass
            return s
        else:
            result = np.array([str(token) for token in doc if token.pos_ == part_of_the_language])
            if(len(result)>1):
                s=''
                s+=result[1]
                return s
    else:
        result = np.array([str(token) for token in doc if token.pos_ == part_of_the_language or token.pos_=='PROPN'])
        subject=''
        for i in doc:
            if i.dep_ == "nsubj":
                subject = i.orth_
        for i in result:
            if(i==subject):
                index = np.argwhere(result==subject)
                result=np.delete(result,index)
        if(part_of_the_language=='NOUN' and len(result)>1): 
            obje=''
            for i in doc:
                if i.dep_ == "dobj":
                    obje = i.orth_
                if(len(str(obje))>0):
                    return obje   
            old_index = np.argwhere(text_array==result[0])
            for i in range(1,len(result)): 
                new_index=np.argwhere(text_array==result[i])
                if(new_index-1==old_index):
                    s=''
                    s+=result[i-1]+' '+result[i]
                    return s
                else:
                    old_index=new_index       
            return result[0]        

        s=''
        for i in result:
            s+=i+' '
            return s      

In [82]:
import json
import cherrypy
from cherrypy import tools

def error_page_404(status, message, traceback, version):
    return "404 Error!"

class HomeController():
    @cherrypy.expose
    @cherrypy.tools.json_out()
    @cherrypy.tools.json_in()
    def GetErrors(self, **kwargs):
        input_json = cherrypy.request.json
        role = input_json["role"].lower()
        feature = input_json["feature"].lower()
        reasons = input_json["reason"].lower()
        f=''
        r=np.array[[]**len(reason)]
        if return_lang_part(feature, nlp, 'VERB')=='':
            f='what is action? '
        if return_lang_part(feature, nlp, 'NOUN')=='':
            f+='what is object?' 
        for i in reasons:    
            if return_lang_part(reasons[i], nlp, 'VERB')=='':
                r[i]='what is action? '
            if return_lang_part(reasons[i], nlp, 'NOUN')=='':
                r[i]+='what is object?'     
        return [role:”  “, feature: [f], reason: [r]]
    
    

    @cherrypy.expose
    @cherrypy.tools.json_out()
    @cherrypy.tools.json_in()
    def GetErrors(self, **kwargs):
        input_json = cherrypy.request.json
        role = input_json["role"].lower()
        feature = input_json["feature"].lower()
        reasons = input_json["reason"].lower()
        return ...

def start_server():
    cherrypy.tree.mount(HomeController(), '/')
    cherrypy.config.update({'error_page.404': error_page_404})
    cherrypy.config.update({'server.socket_port': 8098})
    cherrypy.engine.start()

if __name__ == 'main':
    start_server()


In [84]:
print(requests.post('http://localhost:8099/GetStates', json={'filter': 'ne'}).content)

NameError: name 'requests' is not defined

In [65]:
data = pd.read_csv('dataset_hackaton.csv', sep='|')

data = data.dropna()

data['Feature_verb'] = pd.Series([return_lang_part(data['Feature'][k], nlp, 'VERB') for k in data.index], index=data.index)
data['Feature_obj'] = pd.Series([return_lang_part(data['Feature'][k], nlp, 'NOUN') for k in data.index], index=data.index)
data['Reason_verb'] = pd.Series([return_lang_part(data['Reason'][k], nlp, 'VERB') for k in data.index], index=data.index)
data['Reason_obj'] = pd.Series([return_lang_part(data['Reason'][k], nlp, 'NOUN') for k in data.index], index=data.index)
data=data.dropna()
data

,Story Index,Project Name,User Name,Feature,Reason,Feature_verb,Feature_obj,Reason_verb,Reason_obj
1,0,Instagram,user,Sign up with phone or email,Add New Post,Sign up,phone,Add,Post
2,0,Instagram,user,Sign up with phone or email,Use search,Sign up,phone,Use,search
3,0,Instagram,user,Sign up with phone or email,Add a story,Sign up,phone,Add,story
5,0,Instagram,user,Sign up with phone or email,Go to Like history,Sign up,phone,Go,history
6,0,Instagram,user,Sign up with phone or email,Go to my profile,Sign up,phone,Go,profile
8,0,Instagram,user,Login with Facebook,Add New Post,Login,Facebook,Add,Post
9,0,Instagram,user,Login with Facebook,Use search,Login,Facebook,Use,search
10,0,Instagram,user,Login with Facebook,Add a story,Login,Facebook,Add,story
12,0,Instagram,user,Login with Facebook,Go to Like history,Login,Facebook,Go,history
13,0,Instagram,user,Login with Facebook,Go to my profile,Login,Facebook,Go,profile
